# Block 7 Übung 2: Visualisierung von Vitalparametern

## Bibliotheken & Konfiguration

In [ ]:
packages <- c("readr", "dplyr", "ggplot2", "gridExtra")
install.packages(setdiff(packages, rownames(installed.packages())))
lapply(packages, require, character.only = TRUE)

base_url <- "https://raw.githubusercontent.com/ganslats/TMF-School-2021-Block-7/master/Rohdaten/"

## MIMIC III-Basisdaten laden

In [ ]:
# Patientenstammdaten laden
mimic.patients.raw <- read_csv(paste(base_url, "mimic-iii-demo/PATIENTS.csv", sep=""),
                               col_types = cols(row_id = col_integer(), subject_id = col_integer(), gender = col_character(), dob = col_datetime(format = ""), dod = col_datetime(format = ""), dod_hosp = col_datetime(format = ""), dod_ssn = col_datetime(format = ""), expire_flag = col_double()))

# Behandlungsfälle laden
mimic.admissions.raw <- read_csv(paste(base_url, "mimic-iii-demo/ADMISSIONS.csv", sep=""),
                                 col_types = cols(  row_id = col_integer(), subject_id = col_integer(), hadm_id = col_integer(), admittime = col_datetime(format = ""), dischtime = col_datetime(format = ""), deathtime = col_datetime(format = ""), admission_type = col_character(), admission_location = col_character(), discharge_location = col_character(), insurance = col_character(), language = col_character(), religion = col_character(), marital_status = col_character(), ethnicity = col_character(), edregtime = col_datetime(format = ""), edouttime = col_datetime(format = ""), diagnosis = col_character(), hospital_expire_flag = col_double(), has_chartevents_data = col_double()))

# Intensivaufenthalte laden
mimic.icustays.raw <- read_csv(paste(base_url, "mimic-iii-demo/ICUSTAYS.csv", sep=""),
                               col_types = cols(row_id = col_integer(), subject_id = col_integer(), hadm_id = col_integer(), icustay_id = col_integer(), dbsource = col_character(), first_careunit = col_character(), last_careunit = col_character(), first_wardid = col_double(), last_wardid = col_double(), intime = col_datetime(format = ""), outtime = col_datetime(format = ""), los = col_double()))


## MIMIC III-Chartevents laden

Hinweis: dieser Schritt kann wegen der Größe der Tabelle (>75MB) etwas dauern.

In [ ]:
# Chartevents laden
mimic.chartevents.raw <- read_csv(paste(base_url, "mimic-iii-demo/CHARTEVENTS.csv", sep=""),
                                  col_types = cols(row_id = col_integer(), subject_id = col_integer(), hadm_id = col_integer(), icustay_id = col_integer(), itemid = col_integer(), charttime = col_datetime(format = ""), storetime = col_datetime(format = ""), cgid = col_double(), value = col_character(), valuenum = col_double(), valueuom = col_character(), warning = col_double(), error = col_double(), resultstatus = col_character(), stopped = col_character()))

### Bezeichner ergänzen

In [ ]:
# Bezeichner laden
mimic.d_items.raw <- read_csv(paste(base_url, "mimic-iii-demo/D_ITEMS.csv", sep=""),
                              col_types = cols(row_id = col_integer(), itemid = col_integer(), label = col_character(), abbreviation = col_character(), dbsource = col_character(), linksto = col_character(), category = col_character(), unitname = col_character(), param_type = col_character(), conceptid = col_character()))

# Bezeichner hinzufügen
mimic.chartevents.annotated <- mimic.chartevents.raw %>%
  inner_join(mimic.d_items.raw %>% select(itemid, category, label), by = "itemid")

## Bereitgestelltes Mapping der Vitalparameter auf LOINC laden

In [ ]:
map.vitalparameter <- read_delim(paste(base_url, "%C3%9Cbung/map_chartevents_vitalzeichen.csv", sep=""), delim = ";",
                                 col_types = cols(group_id = col_character(), group_name = col_character(), itemid = col_double(), label = col_character(), loinc_code_generic = col_character(), loinc_name_generic = col_character(), loinc_code_specific = col_character(), loinc_name_specific = col_character()))
head(map.vitalparameter)

## Vitalparameter mit Hilfe des Mappings aus den Chartevents auslesen

In [ ]:
vitalparameter <- mimic.chartevents.annotated %>% 
    select(subject_id, hadm_id, icustay_id, itemid, charttime, valuenum, valueuom, warning, error) %>%
    inner_join(map.vitalparameter, by = "itemid")
head(vitalparameter)

## Aufgabe: Visualisieren Sie Vitalparameterverläufe einzelner Intensiv-Aufenthalte mit ggplot!

Hier können Sie das Wissen aus dem Abschnitt 4, Demo 2 (Daten abfragen mit Tidyverse) und dem Abschnitt 6, Demo 1 (Daten visualisieren, Diagramme) anwenden.
* zugehörige Notebooks: [teil-4-demo-02-abfragen-tidyverse.ipynb](https://github.com/ganslats/TMF-School-2021-Block-7/blob/main/Notebooks/teil-4-demo-02-abfragen-tidyverse.ipynb), [teil-6-demo-01-visualisieren-diagramme.ipynb](https://github.com/ganslats/TMF-School-2021-Block-7/blob/main/Notebooks/teil-6-demo-01-visualisieren-diagramme.ipynb)
* nutzen Sie gerne auch das [ggplot2-Cheat Sheet von RStudio](https://github.com/rstudio/cheatsheets/raw/master/data-visualization-2.1.pdf)

Filtern Sie den oben generierten Vitalparameter-Datensatz auf die Inhalte jeweils eines der folgenden Intensivaufenthalte mit Hilfe der Spalte `icustay_id`:
* 249805
* 276601
* weitere `ìcustay_id`-Nummern können Sie aus dem Tibble `mimic.icustays.raw` entnehmen und sich dabei z.B. an der Länge des Aufenthalts in der Spalte `los` (length of stay) orientieren

Tipps zu ggplot:
* überlegen Sie, welche Variablen Sie für die x- und y-Achsen in der `aes()`-Funktion einsetzen möchten
* Sie können mehrere Variablen im gleichen Diagramm unterscheiden, indem Sie z.B. die Farbe (`fill`-Attribut) über die Variable setzen
* Sie können Variablen auf mehrere (Teil-)Diagramme aufteilen, indem Sie die `facet_wrap()`-Funktion benutzen
* alternativ können Sie auch mehrere Plots getrennt erstellen und über die `grid.arrange()`-Funktion des Pakets gridExtra explizit layouten
* mit der `ylim(von, bis)`-Funktion können Sie festlegen welchen Wertebereich die Y-Skala abdecken soll
* mit der `ylab(titel)`-Funktion können Sie den Titel der y-Achase festlegen

### Stellen Sie alle Vitalparameter in einer gemeinsamen Grafik dar!

### Stellen Sie Blutdruck, Puls & Temperatur in jeweils getrennten Grafiken dar!